This jupyter notebook has been created to show [ipyleaflet] library capabilities with maps. 

The [ipyleaflet] library makes it easy to visualize data that has been manipulated in Python on an interactive [Leaflet.js] map.

It is required to enable [ipywidgets] to interact with this jupyter notebook.

[ipyleaflet]: https://github.com/jupyter-widgets/ipyleaflet
[Leaflet.js]: https://leafletjs.com/
[ipywidgets]: https://ipywidgets.readthedocs.io/en/stable/index.html

In [1]:
import numpy as np  # linear algebra, arrays
import pandas as pd  # data structures
from ipyleaflet import (
    basemap_to_tiles,
    basemaps,
    CircleMarker,
    GeoJSON,
    Heatmap,
    Map,
    Marker,
    MarkerCluster,
    Polyline,
)
from ipywidgets import HTML

Countries selection on World Map

In [2]:
import json
import urllib.request

# world countries boundaries json file
world_countries = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json"

with urllib.request.urlopen(world_countries) as url:
    geo_json_data = json.loads(url.read().decode())


def country_color_function(feature):
    """Countries which id start with 'A' set to green and others to grey."""
    if str(feature["id"]).startswith("A"):
        return "#00f000"
    else:
        return "#808080"


for feature in geo_json_data["features"]:
    feature["properties"]["style"] = {
        "color": country_color_function(feature),
        "weight": 1,
        "fillColor": country_color_function(feature),
        "fillOpacity": 0.25,
    }

world_map = Map(center=(0.0, 0.0), zoom=2)

map_layer = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
world_map.add_layer(map_layer)

geo = GeoJSON(data=geo_json_data)

world_map.add_layer(geo)

world_map

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

Markers on Map

In [3]:
madrid_map = Map(center=(40.4569, -3.7033), zoom=11)

colors_list = [
    "red",
    "blue",
    "green",
    "orange",
    "purple",
    "darkred",
    "lightred",
    "beige",
    "darkblue",
    "darkgreen",
    "cadetblue",
    "darkpurple",
    "white",
    "pink",
    "lightblue",
    "lightgreen",
    "gray",
    "black",
    "lightgray",
]

colors_len = len(colors_list)

# markers
marker_keys = ["name", "location", "icon"]

airport = ["Adolfo Suarez Madrid Barajas Airport", (40.47, -3.57), "plane"]
station = ["Madrid Puerta de Atocha Station", (40.406, -3.69), "ok-sign"]
home = ["Home", (40.5, -3.66), "home"]
museum = ["Museum", (40.44, -3.73), "remove-sign"]

markers_list = []
markers_list.append(dict(zip(marker_keys, airport, strict=True)))
markers_list.append(dict(zip(marker_keys, station, strict=True)))
markers_list.append(dict(zip(marker_keys, home, strict=True)))
markers_list.append(dict(zip(marker_keys, museum, strict=True)))

# add markers to cluster
markers_array = []

for marker in markers_list:
    popup = HTML()
    popup.value = marker["name"]

    markers_array.append(
        Marker(
            location=marker["location"],
            popup=popup,
        )
    )

marker_cluster = MarkerCluster(markers=(markers_array))
madrid_map.add_layer(marker_cluster)

# add table to popup
popup_table = HTML()

df = pd.DataFrame(
    data=[["09:00 to 19:00", "10:00 to 17:00"], ["10:00 to 18:00", "10:00 to 15:00"]],
    columns=["High Season Opening Hours", "Low Season Opening Hours"],
    index=["Monday to Friday", "Weekends & Holidays"],
)

popup_table.value = df.to_html(
    classes="table table-striped table-hover table-condensed table-responsive"
)

marker_table = Marker(location=(40.485, -3.74), popup=popup_table)

madrid_map.add_layer(marker_table)
madrid_map

Map(center=[40.4569, -3.7033], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

Heat Map generated with random data

In [4]:
data = (np.random.normal(size=(500, 3)) * np.array([[5, 5, 5]]) + np.array([[47, 20, 48]])).tolist()

In [5]:
europe_map = Map(center=(47, 5), zoom=4)

map_layer = basemap_to_tiles(basemaps.Esri.WorldTopoMap)
europe_map.add_layer(map_layer)

heatmap = Heatmap(locations=data, radius=50)

europe_map.add_layer(heatmap)

europe_map

Map(center=[47, 5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

Lines and circles on Map

In [6]:
line_keys = ["stations", "points"]

seville_paris = [
    ["Paris", "Barcelona", "Madrid", "Seville"],
    [(48.858093, 2.294694), (41.390205, 2.154007), (40.4169, -3.7033), (37.392529, -5.994072)],
]

paris_brussels = [["Brussels", "Paris"], [(50.8333, 4.35), (48.858093, 2.294694)]]

brussels_berlin = [["Berlin", "Brussels"], [(52.5186, 13.4081), (50.8333, 4.35)]]

paris_london = [
    ["Paris", "Calais", "London"],
    [(51.5072, -0.1275), (50.9475, 1.91), (48.858093, 2.294694)],
]

lines_list = []
lines_list.append(dict(zip(line_keys, paris_brussels, strict=True)))
lines_list.append(dict(zip(line_keys, seville_paris, strict=True)))
lines_list.append(dict(zip(line_keys, brussels_berlin, strict=True)))
lines_list.append(dict(zip(line_keys, paris_london, strict=True)))

# create the map
lines_map = Map(center=(46, 0), zoom=4)

# add lines to map
for i, line in enumerate(lines_list):
    my_PolyLine = Polyline(
        color=colors_list[i % colors_len],
        locations=line["points"],
        opacity=1.0,
        fill=False,
        weight=5,
    )
    lines_map.add_layer(my_PolyLine)

    # add circles to map
    for pos in range(len(line["points"])):
        popup = HTML()
        popup.value = line["stations"][pos]

        my_Circle = CircleMarker(
            location=line["points"][pos],
            radius=6,
            popup=popup,
            fill=True,
            fill_color="white",
            color="black",
            fill_opacity=0.7,
        )
        lines_map.add_layer(my_Circle)

lines_map

Map(center=[46, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

Great Circles on Map

In [7]:
import math


def gcIntermediatePoints(origin, destination, steps):
    """Calculate Intermediate Points of Great Circle.

    Formula:

    f: factor
    d: distance between points

    A = sin((1 - f) * d) / sin(d)
    B = sin(f * d) / sin(d)
    x = A * cos(lat1) * cos(lon1) + B * cos(lat2) * cos(lon2)
    y = A * cos(lat1) * sin(lon1) + B * cos(lat2) * sin(lon2)
    z = A * sin(lat1) + B * sin(lat2)
    lat = atan2(z, sqrt(x ^ 2 + y ^ 2))
    lon = atan2(y, x)

    Source http://www.edwilliams.org/avform.htm

    Args:
        origin [Lat,Lon]: Origin point coordinates in degrees.
        destination [Lat,Lon]: Destination point coordinates in degrees.
        steps (int): Number of steps between origin and destination.

    Returns:
        gcPoints: Return great circle points including origin and destination.

    """

    ori_lat, ori_long = origin
    dest_lat, dest_long = destination

    ori_lat = math.radians(ori_lat)
    ori_long = math.radians(ori_long)
    dest_lat = math.radians(dest_lat)
    dest_long = math.radians(dest_long)

    distance = 2 * math.asin(
        math.sqrt(
            math.pow((math.sin((ori_lat - dest_lat) / 2)), 2)
            + math.cos(ori_lat)
            * math.cos(dest_lat)
            * math.pow(math.sin((ori_long - dest_long) / 2), 2)
        )
    )

    gcPoints = [origin]

    # intermediate points
    for step in range(1, steps):
        fraction = float(step / steps)

        A = math.sin((1 - fraction) * distance) / math.sin(distance)
        B = math.sin(fraction * distance) / math.sin(distance)
        x = A * math.cos(ori_lat) * math.cos(ori_long) + B * math.cos(dest_lat) * math.cos(
            dest_long
        )
        y = A * math.cos(ori_lat) * math.sin(ori_long) + B * math.cos(dest_lat) * math.sin(
            dest_long
        )
        z = A * math.sin(ori_lat) + B * math.sin(dest_lat)
        point_lat = math.atan2(z, math.sqrt(math.pow(x, 2) + math.pow(y, 2)))
        point_lng = math.atan2(y, x)

        gcPoints.append((math.degrees(point_lat), math.degrees(point_lng)))

    gcPoints.append(destination)

    return gcPoints

In [8]:
gc_map = Map(center=(44.0, -30.0), zoom=3)

map_layer = basemap_to_tiles(basemaps.OpenStreetMap.BlackAndWhite)
gc_map.add_layer(map_layer)

city_keys = ["name", "location"]
ny_data = ["New York", (40.488224, -73.131836)]
paris_data = ["Paris", (48.856667, 2.350987)]
madrid_data = ["Madrid", (40.4169, -3.7033)]
london_data = ["London", (51.5072, -0.1275)]
berlin_data = ["Berlin", (52.5186, 13.4081)]

ny = dict(zip(city_keys, ny_data, strict=True))
paris = dict(zip(city_keys, paris_data, strict=True))
madrid = dict(zip(city_keys, madrid_data, strict=True))
london = dict(zip(city_keys, london_data, strict=True))
berlin = dict(zip(city_keys, berlin_data, strict=True))

cities_list = [ny, paris, madrid, london, berlin]

# great circles
ny_paris_gc = gcIntermediatePoints(ny["location"], paris["location"], 10)
ny_madrid_gc = gcIntermediatePoints(ny["location"], madrid["location"], 10)
ny_london_gc = gcIntermediatePoints(ny["location"], london["location"], 10)
ny_berlin_gc = gcIntermediatePoints(ny["location"], berlin["location"], 10)

gc_list = []
gc_list.append(ny_paris_gc)
gc_list.append(ny_madrid_gc)
gc_list.append(ny_london_gc)
gc_list.append(ny_berlin_gc)

# add great circles to map
for i, gc in enumerate(gc_list):
    my_PolyLine = Polyline(color=colors_list[i % colors_len], locations=gc, fill=False, weight=5)
    gc_map.add_layer(my_PolyLine)

# add circles to map
for city in cities_list:
    popup = HTML()
    popup.value = city["name"]

    my_Circle = CircleMarker(
        location=city["location"],
        radius=6,
        popup=popup,
        fill=True,
        fill_color="white",
        color="black",
        fill_opacity=0.7,
    )
    gc_map.add_layer(my_Circle)

gc_map

Map(center=[44.0, -30.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…